# Tokenization

### BertTweet

- fastBPE
- 64K subword

### Twilbert
- SentencePiece (fastBPE)
- 30k subword 

In [76]:
%load_ext autoreload
%autoreload 2
from glob import glob

num_files = 100
tweet_files = glob("../../data/filtered_tweets/*.txt")

train_files = tweet_files[:2]


tweets = list([x.strip("\n") for x in open(tweet_files[0])])[:1_000_000]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
len(tweets)

1000000

In [111]:

SentencePieceBPETokenizer().normalizer

In [89]:
from tokenizers import SentencePieceUnigramTokenizer, SentencePieceBPETokenizer, BertWordPieceTokenizer, ByteLevelBPETokenizer
from tokenizers import normalizers 

tokenizer = SentencePieceBPETokenizer()
#replacement="_")


tokenizer.normalizer = normalizers.Sequence([
    normalizers.Lowercase(),
    normalizers.NFKC()
])

In [110]:
from finetune_vs_scratch.preprocessing import special_tokens
from finetune_vs_scratch.tokenizer import tokenizer_special_tokens

#tokenizer.add_special_tokens(tokenizer_special_tokens)
tokenizer.train_from_iterator(
    tweets,
    vocab_size=30_000,
    min_frequency=,
    show_progress=True,
    limit_alphabet=300,
    special_tokens=tokenizer_special_tokens + special_tokens,
)

## Alphabet

'! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A E I N O S [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ¡ ¬ ¿ à á â ã ä ç è é ê ì í î ï ñ ò ó ô õ ö ù ú ü ć č đ ğ ı ş š ž ɪ ɴ ˈ ̄ ̇ ̶ а в д е и к л м н о р с т у ، ؟ ء آ أ ؤ إ ئ ا ب ة ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ـ ف ق ك ل م ن ه و ى ي ً ٍ َ ُ ِ ّ ْ ٰ ٹ پ چ ڑ ک گ ں ھ ہ ۃ ی ے ۔ \u06dd ۡ ं क ग ज त द न प ब म य र ल स ह ़ ा ि ी ु े ै ो ् ก ค ง ด ต ท น บ ม ย ร ล ว ห อ ั า ี เ แ ่ ้ ᄏ ᅲ ᴀ ᴇ ᴏ \u200b \u200d \u2060 \u2063 \u2066 \u2069 ⃢ ⃣ → ━ ┃ ┓ ┗ ┛ ▁ █ ▪ ▶ ► ☀ ★ ☆ ☇ ☑ ♪ ♫ ⚒ ⚘ ⚠ ⛏ ✍ ✔ ❣ ➡ ➽ ⠀ ⬇ 、 。 「 」 あ い う お か が き く け こ さ し す た だ ち っ つ て で と な に の は ま も ゃ よ ら り る れ を ん ア イ ォ ク コ シ ジ ス タ チ ッ ツ ト ニ ノ バ フ プ マ ャ ラ リ ル レ ロ ン ・ 一 不 么 了 人 你 博 可 哈 啊 大 好 我 描 日 是 最 有 本 爱 王 的 真 가 거 게 결 고 국 그 근 기 김 나 남 내 너 년 는 늘 니 다 단 대 더 데 도 동 두 드 든 들 라 랑 럽 레 로 를 리 마 만 모 몬 무 미 민 박 방 백 번 보 불 븐 비 사 상 생 서 석 성 세 소 수 스 시 아 안 야 어 에 엑 엔 여 열 영 예 오 요 우 운 워 원 유 으 은 을 음 의 이 인 일 있 자 작 정 제 주 준 중 즈 지 직 진 차 찬 축 카 케 크 클 키 타 탄 태 투 트 티 팬 포 

In [85]:
tokenizer.encode("@usuario son UNA MIERDA").tokens

['▁sonr', 'den', '▁claro', '▁arri']

In [112]:
from transformers import PreTrainedTokenizerFast

transformer_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer
)

'@usuario tugo bierno skere comunista'

In [ ]:

transformer_tokenizer.save_pretrained

In [119]:
normalizer = normalizers.BertNormalizer(clean_text=True, strip_accents=True)

normalizer.normalize_str("álaba")

'alaba'